Import dependencies

In [ ]:
%reload_ext autoreload
%autoreload 1
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cobra

from yeast8model import Yeast8Model, biomass_component_list_orig, compare_fluxes

# Construct models of cells of interest, optimise

In [ ]:
glucose_bounds = (-1, 0) # nearer the default
#glucose_bounds = (-18.5, 0) # for E. coli

wt = Yeast8Model("./models/yeast-GEM-BiGG.xml", growth_id="GROWTH", biomass_id="BIOMASS_yeast-GEM")
wt.biomass_component_list = biomass_component_list_orig
wt.add_media_components(['EX_o2_e'])
wt.model.reactions.EX_glc__D_e.bounds = glucose_bounds
wt.solution = wt.optimize()

BY4741 = Yeast8Model("./models/yeast-GEM-BiGG.xml", growth_id="GROWTH", biomass_id="BIOMASS_yeast-GEM")
BY4741.biomass_component_list = biomass_component_list_orig
BY4741.add_media_components(['EX_o2_e'])
BY4741.model.reactions.EX_glc__D_e.bounds = glucose_bounds
BY4741.make_auxotroph("BY4741_orig")
BY4741.solution = BY4741.optimize()

zwf1 = Yeast8Model("./models/yeast-GEM-BiGG.xml", growth_id="GROWTH", biomass_id="BIOMASS_yeast-GEM")
zwf1.biomass_component_list = biomass_component_list_orig
zwf1.add_media_components(['EX_o2_e'])
zwf1.model.reactions.EX_glc__D_e.bounds = glucose_bounds
zwf1.make_auxotroph("BY4741_orig")
zwf1.knock_out_list(["YNL241C"])
zwf1.solution = zwf1.optimize()

tsa2 = Yeast8Model("./models/yeast-GEM-BiGG.xml", growth_id="GROWTH", biomass_id="BIOMASS_yeast-GEM")
tsa2.biomass_component_list = biomass_component_list_orig
tsa2.add_media_components(['EX_o2_e'])
tsa2.model.reactions.EX_glc__D_e.bounds = glucose_bounds
tsa2.make_auxotroph("BY4742_orig")
tsa2.knock_out_list(["YDR453C"])
tsa2.solution = tsa2.optimize()

# Ablate

In [ ]:
wt.ablation_result = wt.ablate()

fig, ax = plt.subplots()
wt.ablation_barplot(ax)
plt.show()

In [ ]:
BY4741.ablation_result = BY4741.ablate()

fig, ax = plt.subplots()
BY4741.ablation_barplot(ax)
plt.show()

In [ ]:
zwf1.ablation_result = zwf1.ablate()
fig, ax = plt.subplots()
zwf1.ablation_barplot(ax)
plt.show()

In [ ]:
tsa2.ablation_result = tsa2.ablate()
fig, ax = plt.subplots()
tsa2.ablation_barplot(ax)
plt.show()

In [ ]:
wt.ablation_result

In [ ]:
BY4741.ablation_result

In [ ]:
zwf1.ablation_result

In [ ]:
tsa2.ablation_result

# Compare fluxes

In [ ]:
diff_fluxes_sorted = compare_fluxes(BY4741, zwf1)

In [ ]:
for rxn_id, flux in diff_fluxes_sorted.items():
    print(f'{rxn_id}, {wt.model.reactions.get_by_id(rxn_id).name}, {flux}')

In [ ]:
diff_fluxes_sorted = compare_fluxes(BY4741, tsa2)

In [ ]:
for rxn_id, flux in diff_fluxes_sorted.items():
    print(f'{rxn_id}, {wt.model.reactions.get_by_id(rxn_id).name}, {flux}')

In [ ]:
diff_fluxes_sorted = compare_fluxes(wt, BY4741)

In [ ]:
for rxn_id, flux in diff_fluxes_sorted.items():
    print(f'{rxn_id}, {wt.model.reactions.get_by_id(rxn_id).name}, {flux}')

In [ ]:
for reaction_id in BY4741.model.medium.keys():
    print(BY4741.model.reactions.get_by_id(reaction_id).name)

In [ ]:
diff_fluxes_sorted = compare_fluxes(wt, BY4741)

In [ ]:
diff_fluxes_sorted = compare_fluxes(wt, wt)

In [ ]:
diff_fluxes_sorted